### robot machine state 공부


### #1

In [1]:
from enum import Enum
import time

class State(Enum):
    Idle = 0
    Move = 1
    Attack = 2

class Monster:
    def __init__(self):
        self._state = State.Idle   # Start()

    def update(self):
        if self._state == State.Idle:
            # Idle 행동 구현...
            print("Idle 행동 실행 중...")
        
        elif self._state == State.Move:
            # Move 행동 구현...
            print("Move 행동 실행 중...")
        
        elif self._state == State.Attack:
            # Attack 행동 구현...
            print("Attack 행동 실행 중...")

    # 테스트용 상태 변경 함수
    def set_state(self, new_state):
        self._state = new_state


# 실행 예시
monster = Monster()

for _ in range(3):   ### "idle 상태가 계속 실행되는 것"의 시뮬레이션. 실제로 ros2 timercallback과 같은 현상
    monster.update()
    time.sleep(1)

monster.set_state(State.Move)
monster.update()

monster.set_state(State.Attack)
monster.update()


Idle 행동 실행 중...
Idle 행동 실행 중...
Idle 행동 실행 중...
Move 행동 실행 중...
Attack 행동 실행 중...


Idle → Move → Attack
조건 a면 공격으로 전환
조건 b면 이동으로 전환

- 조건 기반 규칙 시스템

### #2

In [ ]:
BaseState
 ├── IdleState
 ├── MoveState
 └── AttackState

FSM  ← Monster가 들고 있음

In [ ]:
#basestate.py

from abc import ABC, abstractmethod

class BaseState(ABC):
    def __init__(self, monster):
        self._monster = monster     # C# protected Monster _monster;

    @abstractmethod
    def on_state_enter(self):
        pass

    @abstractmethod
    def on_state_update(self):
        pass

    @abstractmethod
    def on_state_exit(self):
        pass

In [ ]:
#idlestate.py

from BaseState import BaseState

class IdleState(BaseState):
    def __init__(self, monster):
        super().__init__(monster)

    def on_state_enter(self):
        print("Enter Idle")

    def on_state_update(self):
        print("Update Idle")

    def on_state_exit(self):
        print("Exit Idle")

In [ ]:
#movestate.py

from BaseState import BaseState

class MoveState(BaseState):
    def __init__(self, monster):
        super().__init__(monster)

    def on_state_enter(self):
        print("Enter Move")

    def on_state_update(self):
        print("Update Move")

    def on_state_exit(self):
        print("Exit Move")

In [ ]:
#attackstate.py

from BaseState import BaseState

class AttackState(BaseState):
    def __init__(self, monster):
        super().__init__(monster)

    def on_state_enter(self):
        print("Enter Attack")

    def on_state_update(self):
        print("Update Attack")

    def on_state_exit(self):
        print("Exit Attack")


In [ ]:
# fsm.py

class FSM:
    def __init__(self, init_state):
        self._cur_state = init_state
        self.change_state(init_state)   # C# 생성자에서 ChangeState 호출

    def change_state(self, next_state):
        if next_state is self._cur_state:
            return
        
        if self._cur_state is not None:
            self._cur_state.on_state_exit()

        self._cur_state = next_state
        self._cur_state.on_state_enter()

    def update_state(self):
        if self._cur_state is not None:
            self._cur_state.on_state_update()


# monster.py
from FSM import FSM
from IdleState import IdleState
from MoveState import MoveState
from AttackState import AttackState

class Monster:
    def __init__(self):
        self.idle = IdleState(self)
        self.move = MoveState(self)
        self.attack = AttackState(self)

        # 초기 상태 Idle
        self.fsm = FSM(self.idle)

    def update(self):
        self.fsm.update_state()


# 실행 예시
monster = Monster()

monster.update()      # Idle Update
monster.fsm.change_state(monster.move)
monster.update()      # Move Update
monster.fsm.change_state(monster.attack)
monster.update()      # Attack Update